In [6]:
import pandas as pd
import numpy as np
import requests
import json

from pandas.io import gbq
from google.cloud import bigquery,storage
from google.oauth2 import service_account

import gspread

from canvasapi import Canvas

import plotly.express as px

#from instances import env_keys
#API_KEY = env_keys['API_key']
#API_KEY = env_keys['ACCES_TOKEN']
#API_URL = env_keys['API_URL']#+'/accounts/1'

In [7]:
project_id = 'canvas-portal-data-custom'
cred_file = 'canvas-portal-data-custom-6e244db3b826.json'
data_dl = 'data'
scopes = [ "https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file",
            "https://spreadsheets.google.com/auth/spreadsheets"]
credentials = service_account.Credentials.from_service_account_file(cred_file,)

#get Canvas credentials
cred_file_path = '.'
cred_file2 = '{}/instances.json'.format(cred_file_path)
with open(cred_file2,'r') as cred2:
    cred_json = json.load(cred2)

API_KEY = cred_json['ACCES_TOKEN']
API_URL = cred_json['API_URL']#+'/accounts/1'

In [8]:
# Initialize a new Canvas object
canvas = Canvas(API_URL, API_KEY)
canvas.__dict__

{'_Canvas__requester': <canvasapi.requester.Requester at 0x105b542b0>}

### A super quick start to getting information from Canvas API

For more, see:
https://canvasapi.readthedocs.io/_/downloads/en/latest/pdf/

In [ ]:
#Below are some working examples of how to get the user, course etc. from Canvas
#For more, see https://canvasapi.readthedocs.io/_/downloads/en/latest/pdf/
'''
#1. Get the user object
user = canvas.get_user('xys@mit.edu', 'sis_login_id')

    
#2. Get a course by its id or sis_course_id
course = canvas.get_course(1000) #or, course = canvas.get_course('2019SP15.499', 'sis_course_id')
assn = c_.get_assignments()  #get assignments, or get_files() for files
for a_ in assn[:2]:
    print(a_)


#3. Get the account object, then get the courses in this account
accs = canvas.get_accounts()
for acc_course in accs[0].get_courses()[:10]:
    print(acc_course.name)

type_list_ = ['teacher', 'ta', 'designer']
users = course.get_users(enrollment_type=type_list_)
for user in users:
    print(user)

'''

In [5]:
#Get the LT list from the Excel sheet
lt_df_cols = ['School', 'Dept_num', 'Dept_name', 'Name', 'Email']
lt_df = pd.read_excel('Learning_Technologists_updating.xlsx')
lt_df.columns = lt_df_cols
lt_df['Email'] = lt_df.Email.str.lower()
lt_df.tail()

def get_course_list_by_user(user_email):
    try:
        user_ = canvas.get_user(user_email, 'sis_login_id')
        #print(user_.email, user_.name, user_.id)
        course_list = []
        user_courses = user_.get_enrollments(type=['TeacherEnrollment'])
        for uc_ in user_courses:

            #print(uc_, uc_.id, uc_.course_id,'\n\n')

            c_ = canvas.get_course(uc_.course_id)
            assn_ = c_.get_assignments()
            assn_list = []
            for a_ in assn_:
                assn_list.append(a_)
                
                
            course_list.append([(user_.email).lower(), user_.name, c_.name, c_.id, c_.sis_course_id,
                                c_.course_code, c_.created_at, len(assn_list)])
            
            return course_list
    except Exception as e:
        print('Error {} for user {}'.format(e, user_email))
        return None





course_cols = ['Email', 'name', 'course_name', 'course_id','sis_course_id','course_code',
               'created_at', 'num_assignments']
course_list = []


for u_ in lt_df.Email.tolist():
    returned_list = get_course_list_by_user(u_)
    if returned_list != None:
        course_list.extend(returned_list)

course_df = pd.DataFrame.from_records(course_list, columns=course_cols)

Error Not Found for user melc@mit.edu
Error Not Found for user ajzhou@mit.edu


In [7]:
dept_dict = dict(zip(lt_df.Dept_name, lt_df.Dept_num))
courses_to_exclude = [3157, 3158]
course_df['created_at'] = course_df['created_at'].str[:10]
coruse_created_start = '2020-05-25'
course_df_1 = course_df[(~course_df.course_id.isin(courses_to_exclude)) & 
                        (course_df.created_at>=coruse_created_start)].reset_index(drop=True)
course_df_1['if_sandbox'] = np.where(course_df_1.sis_course_id.str.contains('sand-'), 1, 0)
course_df_1.tail()

,Email,name,course_name,course_id,sis_course_id,course_code,created_at,num_assignments,if_sandbox
36,mmpc2000@mit.edu,Mariela Perez-Cabarcas,Temporary Sandbox,5066,sand-mmpc2000-2021FA,mmpc2000,2020-05-28,6,1
37,kmendez@mit.edu,Keegan Mendez,Countdown to OQE 2020,5050,sand-kmendez-2021FA,kmendez,2020-05-28,8,1
38,glasserk@mit.edu,Kaili Glasser,Temporary Sandbox,4999,sand-glasserk-2021FA,glasserk,2020-05-28,5,1
39,murphyl@mit.edu,Lorcan Murphy,Temporary Sandbox,5074,sand-murphyl-2021FA,murphyl,2020-05-28,6,1
40,ajlwill@mit.edu,Anna Williams,Temporary Sandbox,4983,sand-ajlwill-2021FA,ajlwill,2020-05-28,13,1


In [8]:
lt_course_df = lt_df[['Dept_num','Dept_name','Email']].merge(course_df_1, on=['Email'], how='outer')
lt_course_df_no_sandbox = lt_course_df[lt_course_df.if_sandbox==0]
lt_course_df.to_csv('lt_course_df.csv', index=None)
lt_course_df_no_sandbox.to_csv('lt_course_df_no_sandbox.csv', index=None)
lt_course_df.drop_duplicates(inplace=True)
lt_course_df = lt_course_df.reset_index(drop=True)
lt_course_df.to_gbq('lt_courses.courses_by_lt_all', project_id, if_exists='replace', 
                               credentials=credentials)

lt_grouped = lt_course_df[lt_course_df.if_sandbox==0].groupby(['Email','name','Dept_num', 'Dept_name'], 
                                  as_index=False).agg({'course_id':'count'})
lt_grouped

dept_df_no_sandbox = lt_course_df_no_sandbox.groupby('Dept_name',as_index=False).agg({'course_id':'count'})
dept_df_all = lt_course_df.groupby('Dept_name',as_index=False).agg({'course_id':'count'})
dept_df_no_sandbox.columns = ['Dept_name','num_no_sandbox_courses']
dept_df_all.columns =  ['Dept_name','num_all_courses']
dept_df = dept_df_no_sandbox.merge(dept_df_all, on='Dept_name', 
                        how='outer').fillna(0).sort_values(by='num_no_sandbox_courses', ascending=False)
dept_df['Dept_num'] = dept_df.apply(lambda row: dept_dict[row['Dept_name']], axis=1)
dept_df = dept_df[['Dept_num', 'Dept_name', 'num_no_sandbox_courses','num_all_courses']]
dept_df.to_csv('dept_df.csv', index=None)

#send files to gbq as tables:
dept_df.to_gbq('lt_courses.dept_tally', project_id, if_exists='replace', credentials=credentials)
lt_course_df_no_sandbox.to_gbq('lt_courses.courses_by_lt_no_sandbox', project_id, if_exists='replace', 
                               credentials=credentials)
lt_course_df.to_gbq('lt_courses.courses_by_lt_all', project_id, if_exists='replace', 
                               credentials=credentials)

1it [00:06,  6.54s/it]
1it [00:03,  3.14s/it]
1it [00:03,  3.15s/it]


In [15]:
#Get all courses:

accs = canvas.get_accounts()
course_rows = []
course_cols = ['course_id', 'course_code', 'course_name', 'account_id', 'created_at']
for acc_course in accs[0].get_courses()[:10000]:
    course_rows.append([acc_course.id, acc_course.course_code, acc_course.name, acc_course.account_id, 
                        acc_course.created_at])
    
courses_df = pd.DataFrame.from_records(course_rows, columns=course_cols)
courses_df

,course_id,course_code,course_name,account_id,created_at
0,1,welcome-to-canvas,Welcome to Canvas,1,2016-01-07T16:30:12Z
1,16,GSL,GSL Canvas Pilot,15,2016-10-03T14:39:27Z
2,17,21G.067/21G.597,Digital Media in Japan and Korea,11,2016-10-03T19:42:20Z
3,18,21G.614,Russian IV,14,2016-10-11T13:39:46Z
4,20,21G.501,Japanese I (Section 1),11,2016-10-11T13:55:59Z
...,...,...,...,...,...
4870,5791,bry:2,Temporary Sandbox,139,2020-08-12T12:28:23Z
4871,5792,21M.302 L02,"21M.302 Harmony and Counterpoint II, Lecture 2...",116,2020-08-12T14:59:01Z
4872,5793,cwalley,Temporary Sandbox,139,2020-08-12T16:14:14Z
4873,5794,tbdavies,Temporary Sandbox,139,2020-08-12T16:14:15Z


In [18]:
courses_df[courses_df.course_code.str.contains('21G.503')]

,course_id,course_code,course_name,account_id,created_at
57,87,21G.503,Japanese III,8,2017-08-30T23:31:59Z
201,479,21G.503,Japanese III,8,2018-05-21T16:04:37Z
1024,1778,21G.503,Japanese III,8,2019-08-21T19:17:13Z
3200,3989,21G.503,21G.503 Japanese III [FA20],113,2020-05-28T02:22:49Z


In [20]:
courses_df[(courses_df.created_at>'2020-05-20') & (courses_df.account_id!=139)]

,course_id,course_code,course_name,account_id,created_at
2370,3154,LTI_FBF_SB,LTI_FeedbackFruits_Sandbox,76,2020-05-21T18:14:29Z
2371,3155,CNV.VIS-test1,Visibility Testing,86,2020-05-26T13:11:55Z
2372,3156,CNV-RESOURCES,MIT Canvas Resources for Instructors,15,2020-05-26T14:08:15Z
2373,3157,LearnTechCommunity,Learning Technologist Community,15,2020-05-26T14:12:08Z
2374,3158,LearnTechResources,Learning Technologist Resources,15,2020-05-26T14:14:29Z
...,...,...,...,...,...
4865,5786,10.01.01,10.01.01 Ethics for Engineers,97,2020-08-11T17:53:21Z
4866,5787,10.01.02,10.01.02 Ethics for Engineers,97,2020-08-11T17:53:28Z
4867,5788,10.01.03,10.01.03 Ethics for Engineers,97,2020-08-11T17:53:38Z
4868,5789,10.01.04,10.01.04 Ethics for Engineers,97,2020-08-11T17:53:45Z


In [24]:
sub_account_dict = {}
acc = canvas.get_account(1)
sub_accs = acc.get_subaccounts(recursive=True)
for a_ in sub_accs:
    sub_account_dict[a_.id] = a_.name


In [30]:
sub_account_dict

{2: 'Canvas Demo Courses',
 8: 'Global Studies and Languages',
 15: 'Manually-Created Courses',
 17: 'Sloan School of Management',
 83: 'Non-Sloan Courses Imported by IS&T',
 86: 'Canvas Testing',
 87: 'School of Architecture and Planning',
 92: 'School of Engineering',
 108: 'School of Humanities, Arts, and Social Sciences',
 122: 'School of Science',
 129: 'Non-Academic Departments',
 131: 'Other Academic Departments',
 139: 'Sandboxes',
 9: 'Spanish',
 10: 'Chinese',
 11: 'Japanese',
 12: 'Portuguese',
 13: 'German',
 14: 'Russian',
 80: 'Teamplates',
 20: 'EMBA',
 21: 'Sloan Fellows',
 22: 'Feedback Box  Pilot',
 25: 'Executive Electives',
 26: 'MFIN',
 27: 'LGO',
 28: 'Panopto Pilot',
 29: 'HBSP',
 30: 'MBA',
 31: 'MBAN',
 32: 'SDM',
 33: 'UGRAD',
 35: 'REAP',
 36: 'Open Membership',
 37: 'TestingBluePrint',
 39: 'Sloan Innovation Period',
 40: 'LCMS Pilot',
 76: 'Sloan Sandboxes',
 81: 'Non-Canvas Courses',
 88: '4 - Department of Architecture',
 89: '11 - Department of Urban Stu

In [36]:
excl_acc = [1,17,76]
courses_df_dep_excluded = courses_df[~courses_df.account_id.isin(excl_acc)]
try:
    courses_df_dep_excluded['Dept'] = courses_df_dep_excluded.apply(lambda 
                                            row: sub_account_dict[row['account_id']], axis=1)
except:
    pass
courses_df_dep_excluded

/Users/anindyaroy/Sites/fun/venv/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,course_id,course_code,course_name,account_id,created_at,Dept
1,16,GSL,GSL Canvas Pilot,15,2016-10-03T14:39:27Z,Manually-Created Courses
2,17,21G.067/21G.597,Digital Media in Japan and Korea,11,2016-10-03T19:42:20Z,Japanese
3,18,21G.614,Russian IV,14,2016-10-11T13:39:46Z,Russian
4,20,21G.501,Japanese I (Section 1),11,2016-10-11T13:55:59Z,Japanese
5,21,21G.804,Portuguese IV,12,2016-10-11T13:58:21Z,Portuguese
...,...,...,...,...,...,...
4870,5791,bry:2,Temporary Sandbox,139,2020-08-12T12:28:23Z,Sandboxes
4871,5792,21M.302 L02,"21M.302 Harmony and Counterpoint II, Lecture 2...",116,2020-08-12T14:59:01Z,21M - Music and Theater Arts
4872,5793,cwalley,Temporary Sandbox,139,2020-08-12T16:14:14Z,Sandboxes
4873,5794,tbdavies,Temporary Sandbox,139,2020-08-12T16:14:15Z,Sandboxes


In [40]:
#courses_df[courses_df.account_id==1]
begin_date = '2020-05-01'
filtered_courses_df = courses_df_dep_excluded[courses_df_dep_excluded.created_at>=begin_date]

filtered_courses_df.to_csv('all_canvas_since_{}.csv'.format(begin_date), index=None)
filtered_courses_df

,course_id,course_code,course_name,account_id,created_at,Dept
2361,3145,CNV-SANDBOX,Canvas Core Team Sandbox,86,2020-05-05T16:03:19Z,Canvas Testing
2362,3146,CNV-GROW,Growing with Canvas,86,2020-05-05T16:05:38Z,Canvas Testing
2363,3147,Admin,Admin Guide,15,2020-05-08T18:13:52Z,Manually-Created Courses
2364,3148,IntroCDO_MBA22,Career Foundations - Incoming MBA Class of 2022,36,2020-05-11T19:13:28Z,Open Membership
2365,3149,Template1,Template 1,2,2020-05-12T16:17:55Z,Canvas Demo Courses
...,...,...,...,...,...,...
4870,5791,bry:2,Temporary Sandbox,139,2020-08-12T12:28:23Z,Sandboxes
4871,5792,21M.302 L02,"21M.302 Harmony and Counterpoint II, Lecture 2...",116,2020-08-12T14:59:01Z,21M - Music and Theater Arts
4872,5793,cwalley,Temporary Sandbox,139,2020-08-12T16:14:14Z,Sandboxes
4873,5794,tbdavies,Temporary Sandbox,139,2020-08-12T16:14:15Z,Sandboxes


In [44]:
filtered_courses_df[filtered_courses_df.account_id!=139].groupby('Dept', 
                    as_index=False).agg({'course_code':'count'}).sort_values(by='course_code', ascending=False)

,Dept,course_code
22,6 - Department of Electrical Engineering and C...,136
20,4 - Department of Architecture,128
2,11 - Department of Urban Studies and Planning,105
15,21M - Music and Theater Arts,98
1,10 - Department of Chemical Engineering,96
8,2 - Department of Mechanical Engineering,85
7,18 - Department of Mathematics,82
12,21G - Global Studies and Languages,77
3,"12 - Department of Earth, Atmospheric, and Pla...",76
23,7 - Department of Biology,75
